In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url of the website's ranking database

In [11]:
SITE = "https://www.timeshighereducation.com"
URL = SITE+"/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json"

we access the database and look what is inside

In [12]:
N = 200
r = requests.get(URL)
all_data = r.json()
print(list(all_data.keys()))
data = all_data.get("data")[:N]
print(list(data[0].keys()))
names = [u.get("name") for u in all_data.get("data")]
print("Extracted %d names" % len(names))

['data', 'pillars', 'subjects', 'locations']
['name', 'rank', 'scores_research', 'scores_teaching_rank', 'scores_overall', 'scores_industry_income', 'stats_student_staff_ratio', 'scores_research_rank', 'scores_international_outlook', 'scores_citations_rank', 'member_level', 'stats_female_male_ratio', 'rank_order', 'stats_number_students', 'subjects_offered', 'stats_pc_intl_students', 'record_type', 'aliases', 'url', 'scores_international_outlook_rank', 'scores_overall_rank', 'scores_citations', 'scores_industry_income_rank', 'scores_teaching', 'nid', 'location']
Extracted 1102 names


We want to see which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students. And we want to also do it by aggregating by country and region. To do that we observe that both information are already present in the database (already computed in "stats_pc_intl_students", and "stats_student_staff_ratio")

In [13]:
data[37]

{'aliases': 'École Polytechnique Fédérale de Lausanne',
 'location': 'Switzerland',
 'member_level': '0',
 'name': 'École Polytechnique Fédérale de Lausanne',
 'nid': 510,
 'rank': '=38',
 'rank_order': '380',
 'record_type': 'master_account',
 'scores_citations': '94.2',
 'scores_citations_rank': '62',
 'scores_industry_income': '76.0',
 'scores_industry_income_rank': '108',
 'scores_international_outlook': '98.7',
 'scores_international_outlook_rank': '5',
 'scores_overall': '75.3',
 'scores_overall_rank': '380',
 'scores_research': '66.8',
 'scores_research_rank': '54',
 'scores_teaching': '58.8',
 'scores_teaching_rank': '54',
 'stats_female_male_ratio': '28 : 72',
 'stats_number_students': '9,928',
 'stats_pc_intl_students': '55%',
 'stats_student_staff_ratio': '11.2',
 'subjects_offered': 'Architecture,Biological Sciences,Chemical Engineering,Chemistry,Civil Engineering,Communication & Media Studies,Computer Science,Electrical & Electronic Engineering,General Engineering,Geology,

for now not needed -> remove if still the case later

In [14]:
s = requests.Session()
reqs = [(req_id, s.prepare_request(requests.Request('GET', SITE+entry["url"]))) for req_id, entry in enumerate(data)]
print(len(reqs), "requests to be sent.")

resps = [(req_id, s.send(req)) for req_id, req in reqs]

done = [(req_id, resp.text) for req_id, resp in resps if resp.status_code == 200]
failed = [(req_id, resp) for req_id, resp in resps if resp.status_code != 200]

print("%d done, %d failed." % (len(done), len(failed)))

200 requests to be sent.
200 done, 0 failed.


We clean the dataframe by first renaming the columns:

In [15]:
uni = pd.DataFrame(data, columns=["name", "rank", "location", "stats_pc_intl_students", "stats_student_staff_ratio", "url"])
uni.rename(columns={"location":"country","stats_pc_intl_students":"pc_intl_students(%)", "stats_student_staff_ratio":"student_staff_ratio"},inplace=True)

Then, we format both ranks and ratios as numbers

In [16]:
uni["rank"] = uni["rank"].str.extract('(\d+)', expand=False).astype(int)
uni["pc_intl_students(%)"]= uni["pc_intl_students(%)"].str.extract('(\d+)', expand=False).astype(int)

and check that the database has all the values

In [17]:
uni.isnull().values.any()

False

save the dataframe in a file

In [21]:
import pickle

uni.to_pickle("site2.pkl")
pickle.dump(names, open('site2_names.pkl', 'wb'))